In [ ]:
from datascience import *
import numpy as np

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Lecture 10 ##

## Function to Predict

In [ ]:
insurance = Table.read_table('insurance_charges.csv')
insurance

In [ ]:
insurance.scatter('bmi', 'charges')

In [ ]:
insurance.scatter('bmi', 'charges', group = 'smoker')

In [ ]:
insurance_smoker = insurance.where('smoker', 'yes')
insurance_smoker.scatter('bmi', 'charges')

### For a patient with a BMI of 35, how much will the charges be?

In [ ]:
insurance_smoker.scatter('bmi', 'charges')
plots.plot([35, 35], [15000, 65000], color='red', lw=2);

In [ ]:
insurance_smoker.where('bmi', are.equal_to(35))

### Can we use all BMIs *close* to 35? 

In [ ]:
insurance_smoker.scatter('bmi', 'charges')
plots.plot([34, 34], [15000, 65000], color='gold', lw=2);
plots.plot([36, 36], [15000, 65000], color='gold', lw=2);

In [ ]:
nearby = insurance_smoker.where('bmi', are.between(34, 36))
nearby_mean = np.mean(nearby.column('charges'))
nearby_mean

In [ ]:
insurance_smoker.scatter('bmi', 'charges')
plots.plot([34, 34], [15000, 65000], color='gold', lw=2);
plots.plot([36, 36], [15000, 65000], color='gold', lw=2);
plots.scatter(35, nearby_mean, color='red', s=50);

### Can we write a function to do this for all BMIs in the data? 

In [ ]:
def predict(bmi):
    nearby = insurance_smoker.where('bmi', are.between(bmi -1 , bmi + 1))
    return np.mean(nearby.column('charges'))

In [ ]:
predict(35)

In [ ]:
predict(23)

In [ ]:
predict(27)

In [ ]:
predicted_charges = insurance_smoker.apply(predict, 'bmi')
predicted_charges

In [ ]:
insurance_smoker = insurance_smoker.with_column(
    'Predicted charges', predicted_charges)

In [ ]:
insurance_smoker.select(
    'charges', 
    'bmi', 
    'Predicted charges'
).scatter('bmi')

## Grouping by One Column ##

In [ ]:
cereal = Table.read_table('Cereal.csv')
cereal

In [ ]:
cereal.group('Company')

In [ ]:
cereal.select('Company', 'Calories').group('Company', np.average)

In [ ]:
cereal.drop('Serving', 'Name').group('Company', np.average)

In [ ]:
cereal.drop('Serving').group('Company', np.average)

In [ ]:
cereal.select('Company', 'Fat').group('Company', max)

## Grouping By One Column ##

In [ ]:
sleep = Table.read_table('SleepStudy.csv').select(
    'ClassYear',
    'LarkOwl',
    'GPA', 
    'PoorSleepQuality', 
    'AverageSleep', 
    'Drinks'
)

sleep

In [ ]:
sleep.group('ClassYear')

In [ ]:
sleep.group('ClassYear', np.average)

In [ ]:
by_Drinks = sleep.group('Drinks', np.average)
by_Drinks.show()

In [ ]:
# Drop the 'ClassYear average' and LarkOwl average' columns

by_Drinks.drop(1,2).plot('Drinks')

## Grouping by Two Columns ##

In [ ]:
sleep.group(['ClassYear','LarkOwl']).show()

In [ ]:
sleep.group(['ClassYear','LarkOwl'], np.average).show()

## Pivot Tables

In [ ]:
sleep.pivot('ClassYear', 'LarkOwl')

In [ ]:
sleep.pivot('ClassYear', 'LarkOwl', values='Drinks', collect=np.average)